In [ ]:
# NumPy : librairie de calcul scientifique
import numpy as np
# Pandas : librairie de manipulation de données
import pandas as pd
# Pandas : librairie de manipulation de données
import matplotlib.pyplot as plt
# SeaBorn : librairie de graphiques avancés
import seaborn as sns

In [ ]:
df =  pd.read_csv('../input/voicegender/voice.csv')
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

Beside "Label", which is object, all the categories are floats

In [ ]:
df.head(3)

In [ ]:
df.isnull().values.any()


### We have 0 null values

In [ ]:
#Distribution of target variables

colors=['Red', 'Green']
data_y = df[df.columns[-1]]
plt.pie(data_y.value_counts(),colors=colors,labels=['female','male'])
plt.axis('equal')        
print (df['label'].value_counts())  
        
        

In [ ]:
#Box plot see comparaison in lables by other features
df.boxplot(column= 'meanfreq', 
           by= 'label', grid=True)

Les diagrammes en boîte (ou boîtes à moustaches ou box plot) résument quelques caractéristiques de position du caractère étudié (médiane, quartiles, minimum, maximum ou déciles). Ce diagramme est utilisé principalement pour comparer un même caractère dans deux populations de tailles différentes. Il s'agit de tracer un rectangle allant du premier quartile au troisième quartile et coupé par la médiane. On ajoute alors des segments aux extrémités menant jusqu'aux valeurs extrêmes.

In [ ]:
# Plot correlation matrix
correlation = df.corr()
sns.heatmap(correlation)
plt.show()

In [ ]:
#the heat map helps us to see how the feature are similaire to each other : 
##the lighter spots are the most similaire, the oposite can be said about darker spots
### by deleting the features who are too similaire we can have a more accurate results

# Machine learning :

In [ ]:
# train_test_split is responsible to split the datainto (train and test data)
from sklearn.model_selection import train_test_split
# importing Linear Regression Classifier
from sklearn.linear_model import LogisticRegression
# importing Baysian Classifier
from sklearn.naive_bayes import GaussianNB
# importing Decision Trees Classifier
from sklearn.tree import DecisionTreeClassifier
# importing Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
# importing the classification report results
from sklearn.metrics import classification_report
# import confusion matrix
from sklearn.metrics import confusion_matrix
# import metrix to calculate accuracy
from sklearn import metrics, neighbors
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

Spliting data frame into Data and Class(X,y) and converting them into NumpyArray

First test

In [ ]:
#Splitting dataset into (Train, test)
X = df[df.columns[:-1]].values
#y = df['Lable']
y = df[df.columns[-1]].values
# 70-30% of train and test
Xtrain ,Xtest, ytrain ,ytest = train_test_split(X, y, test_size=0.30)


In [ ]:
#Random forest
rand_forest = RandomForestClassifier()
rand_forest.fit(Xtrain, ytrain)
y_pred = rand_forest.predict(Xtest)

In [ ]:
print(metrics.accuracy_score(ytest, y_pred))

In [ ]:
#we have got an accuracy of 98%, which is good, but we still have to clean the data for optimule results.


In [ ]:
print(confusion_matrix(ytest, y_pred))  

In [ ]:
from sklearn.model_selection import cross_val_score
CVFirst = GaussianNB()
CVFirst = CVFirst.fit(Xtrain, ytrain)
test_results = cross_val_score(CVFirst, X, y, cv=10, scoring='accuracy')
print ('On utilisant 10 colonnes. La précision obtenue à partir de l analyse de validation croisée est la suivante  :',test_results.mean())

# Data Cleaning
    

## After doing some rsearch we found out that the average range of a fundamental freq for a human is between 0.085 Khz and 0.255 Khz, and hence we will identify the variable which has this frequency info and remove them assuming it is to be an outlier based on domaine knowledge.
## According to the research, we can say that typical adult male will have a fundamental frequency from 85 Hz to 180 Hz and tipical adult female from 165 Hz to 255 hz 







In [ ]:
male_funFreq_outlier_index =df[((df['meanfun'] < 0.085) | (df['meanfun'] > 0.180)) & 
                                (df['label'] == 'male')].index
female_funFreq_outlier_index =df[((df['meanfun'] < 0.165) | (df['meanfun'] > 0.255)) & 
                                (df['label'] == 'female')].index

In [ ]:
index_to_remove = list(male_funFreq_outlier_index) + list(female_funFreq_outlier_index)
len(index_to_remove)

In [ ]:
data_x=df[df.columns[0:20]].copy()
data2 = data_x.drop(['kurt', 'centroid', 'dfrange'],axis=1).copy()
data2.head(3)
data2= data2.drop(index_to_remove,axis=0)
 
    #Second test

#y = df['Lable']
data_y = pd.Series(y).drop(index_to_remove,axis=0)
Xtrain, Xtest, ytrain, ytest = train_test_split(data2,data_y, test_size=0.30 )
clf1 = RandomForestClassifier()
clf1.fit(Xtrain, ytrain)
y_pred = clf1.predict(Xtest)
print(metrics.accuracy_score(ytest, y_pred))

In [ ]:
clf2 = DecisionTreeClassifier()
clf2.fit(Xtrain, ytrain)
y_predict = clf2.predict(Xtest)
print(metrics.accuracy_score(ytest, y_predict))


In [ ]:
clf3 = GaussianNB()
clf3 = clf3.fit(Xtrain, ytrain)
y_predd = clf3.predict(Xtest)
print(metrics.accuracy_score(ytest,y_predd))

In [ ]:
###Cross validation with same classifire as first time 

test_results = cross_val_score(clf3, data2, data_y, cv=10, scoring='accuracy')
print('On utilisant 10 colonnes. La précision obtenue à partir de l analyse de validation croisée est la suivante :', test_results.mean())

In [ ]:
###Cross validation on the best results :

test_results = cross_val_score(clf2, data2, data_y, cv=10, scoring='accuracy')
print('On utilisant 10 colonnes. La précision obtenue à partir de l analyse de validation croisée est la suivante :', test_results.mean())

confusion matrix

In [ ]:
import pylab as pl
labels = ['female', 'male']
cm = confusion_matrix(ytest, y_pred, labels) #Ypred for RandomForest
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
pl.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
pl.xlabel('Predited')
pl.ylabel('True')
pl.show()

In [ ]:
print(confusion_matrix(ytest, y_pred))  

In [ ]:
print(classification_report(ytest, y_pred))

In [ ]:
#Distribution of male and female 
sns.FacetGrid(df, hue='label', height=5).map(sns.kdeplot, "meanfun").add_legend()
plt.show()

In [ ]:
# Flate-clustering

from sklearn.cluster import KMeans
from matplotlib import style
style.use("ggplot")

data_x = np.array(df[['meanfreq', 'meanfun']])
kmeans = KMeans(n_clusters=2)
kmeans.fit(data_x)

centroids = kmeans.cluster_centers_
labels = kmeans.labels_
 

colors = ["b.", "g."] #Vert = female ; Bleu : male

for i in range(len(data_x)):
    plt.plot(data_x[i][0], data_x[i][1], colors[labels[i]], markersize = 10)
    
plt.scatter(centroids[:, 0], centroids[:, 1], marker= "x", s=100, linewidths = 5, zorder = 10)
plt.ylabel('meanfun')
plt.xlabel('meanfreq')

plt.show()